In [3]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [4]:
SEASONS = list(range(2023, 2027))

In [5]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

os.makedirs(STANDINGS_DIR, exist_ok=True)
os.makedirs(SCORES_DIR, exist_ok=True)

In [8]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range (1, retries+1):
        time.sleep(sleep * i)

        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"TimeoutError on {url}")
            continue
        else:
            break
    return html

In [6]:
async def scrape_season(season):
    url = f"https://basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standings_pages = [f"https://basketball-reference.com{l}" for l in href]

    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [7]:
for season in SEASONS:
    await scrape_season(season)

NameError: name 'get_html' is not defined

In [9]:
standing_files = os.listdir(STANDINGS_DIR)

In [10]:
async def scrape_game(standings_file):

    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores = [f"https://basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [ ]:
for f in standing_files:
    filepath = os.path.join(STANDINGS_DIR, f)

    await scrape_game(filepath)

In [12]:
standing_files

['NBA_2023_games-february.html',
 'NBA_2026_games-november.html',
 'NBA_2026_games-january.html',
 'NBA_2025_games-april.html',
 'NBA_2025_games-november.html',
 'NBA_2026_games-march.html',
 'NBA_2026_games-december.html',
 'NBA_2025_games-december.html',
 'NBA_2026_games-april.html',
 'NBA_2026_games-october.html',
 'NBA_2025_games-may.html',
 'NBA_2023_games-november.html',
 'NBA_2025_games-october.html',
 'NBA_2024_games-february.html',
 'NBA_2025_games-february.html',
 'NBA_2024_games-march.html',
 'NBA_2023_games-june.html',
 'NBA_2023_games-january.html',
 'NBA_2024_games-may.html',
 'NBA_2024_games-june.html',
 'NBA_2026_games-february.html',
 'NBA_2023_games-october.html',
 'NBA_2023_games-december.html',
 'NBA_2025_games-june.html',
 'NBA_2024_games-november.html',
 'NBA_2024_games-october.html',
 'NBA_2025_games-january.html',
 'NBA_2023_games-march.html',
 'NBA_2023_games-may.html',
 'NBA_2025_games-march.html',
 'NBA_2023_games-april.html',
 'NBA_2024_games-december.html',